When you have a mixture in <strong>PyChemkin</strong>, not only can you obtain its properties, you can also extract reaction rate information from it. The mixture can be one that is set up from scratch, or obtained from certain mixture operations, or a point (time or grid) solution of a reactor simulation. The mixture rate utilities let you access the <u>net production rate of species</u> (ROP), the <u>forward and the reverse reaction rates per reaction</u> (RR), and the <u>net chemical heat release rate</u> (HRR). 

This tutorial shows you how to use these <em>Chemkin</em> mixture rate tools and how you can derive useful information from the raw data such as isolating dominant reaction at different mixture conditions.  

In [ ]:
import os
import chemkin as ck                      # Chemkin
import numpy as np                        # number crunching
%matplotlib inline
import matplotlib.pyplot as plt           # plotting

# check working directory
current_dir = os.getcwd()
print('current working directory: ' + current_dir)
# set verbose mode
ck.verbose = True

<h5>Set up the <em>Chemistry Set</em></h5>Create and preprocess the <em>Chemistry</em> set object for the current PyChemkin project. Here the GRI 3.0 mechanism for methane combustion is used to instantiate the <code>MyGasMech</code> <em>Chemistry</em> object.

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = ck.ansys_dir + '\\reaction\\data'
mechanism_dir = data_dir
# create a chemistry set based on GRI 3.0
MyGasMech = ck.Chemistry(label='GRI 3.0')
# set mechanism input files
# inclusion of the full file path is recommended
MyGasMech.chemfile = mechanism_dir+'\\grimech30_chem.inp'
MyGasMech.thermfile = mechanism_dir+'\\grimech30_thermo.dat'
MyGasMech.tranfile = mechanism_dir+'\\grimech30_transport.dat'
# preprocess the mechanism files
iError = MyGasMech.preprocess()

<h5>Create the fuel-air mixture</h5>Set up a stoichiometric methane-air mixture <code>premixed</code> using the <i><u>equivalence ratio</u></i> method. 

In [ ]:
# create a premixed fuel-oxidizer mixture by assigning the equivalence ratio
# create the fuel mixture
fuelmixture = ck.Mixture(MyGasMech)
# set fuel composition
fuelmixture.X = [('CH4', 1.0)]
# setting pressure and temperature is not required in this case
fuelmixture.pressure = 5.0 * ck.Patm
fuelmixture.temperature = 1500.0
# create the oxidizer mixture: air
air = ck.Mixture(MyGasMech)
air.X = [('O2', 0.21), ('N2', 0.79)]
# setting pressure and temperature is not required in this case
air.pressure = 5.0 * ck.Patm
air.temperature = 1500.0
# create the premixed mixture to be defined
premixed = ck.Mixture(MyGasMech)
# products from the complete combustion of the fuel mixture and air
products = ['CO2', 'H2O', 'N2']
# species mole fractions of added/inert mixture. can also create an additives mixture here
add_frac = np.zeros(MyGasMech.KK, dtype=np.double)   # no additives: all zeros
iError = premixed.XbyEquivalenceRatio(MyGasMech, fuelmixture.X, air.X, add_frac, products, equivalenceratio=1.0)
if iError > 0:
    raise RuntimeError

Verify the molar composition of <code>premixed</code>.

In [ ]:
# list the composition of the premixed mixture
premixed.listcomposition(mode='mole')

Set the mixture pressure and temperature to 5 atm and 1600K, respectively. 

In [ ]:
# temperature and pressure are requires to compute the reaction rates
premixed.pressure = 5.0 * ck.Patm
premixed.temperature = 1600.0

<h5>Evaluate the species rate of production</h5>Get the net production rates of <u>all</u> species by using the <em>Mixture</em> method <code>ROP</code> and store thhe rates to <code>rop</code>. The ROP values obtained by <code>ROP</code> are in "mole/cm<sup>3</sup>-sec", alternatively you can use <code>massROP</code> method to get mass-unit ROP values in "g/cm<sup>3</sup>-sec".  Use the optional parameter <code>threshold</code> to get only the ROPs with absolute value above the given threshold value. For example, <code>premixed.ROP(threshold=1.0e-8)</code> will return a <code>rop</code> array in which any raw ROP value with absolute value less than 1.0e-8 is set to zero. By default <code>threshold=0.0</code>, that is, all raw ROP values will be returned.  

In [ ]:
# get the net species molar rates of production [mole/cm3-sec]
rop = premixed.ROP()

List all molar ROP values greater than the threshold value (=0.0 by default) in descending order. <code>species_rates</code> contains the sorted ROP values in descending order and <code>specrate_order</code> has the corresponding species indices in the same ordering. Note that Python list/array is 0-based, you need to increase the integer values in the <code>specrate_order</code> by 1 to get the actual species index which is 1-based. 

In [ ]:
# list the nonzero rates in descending order
specrate_order, species_rates = premixed.listROP()

<h5>Obtain the raw reaction rates</h5>Use method <RxnRates</code> to get the forward (<code>kf</code>) and the reverse (<code>kr</code>) rates of each reaction at the given mixture condition. The rates are in "mole/cm<sup>3</sup>-sec". Only the reverse rates <code>kr</code> are printed here.

In [ ]:
# get the forward and the reverse rates of each reaction
kf, kr = premixed.RxnRates()
print()
print(f'reverse reaction rates: (raw values of all {MyGasMech.IIGas:d} reactions)')
print(str(kr))

Use the <code>listractionrates</code> method to sort and list non-zero net reaction rates in descending order. The net reaction rate is defined by <u>forward rate</u> - <u>reverse rate</u>. You can also use the optional parameter <code>threshold</code> to filter out small reaction rate values. The sorted net reaction rates is stored in <code>net_rxn_rates_1600</code> and the reaction indices in <code>rxn_order</code>.

In [ ]:
# list the nonzero net reaction rates
rxn_order, net_rxn_rates_1600 = premixed.listreactionrates()

Since the Python list/array is 0-based, you need to increment the values in <code>rxn_order</code> by 1 and convert the integer value to a string (stored in <code>orderstr_1600</code>). 

In [ ]:
# convert reaction # from integers to strings
orderstr_1600 = []
for i in range(len(rxn_order)):
    # the array index starting from 0 so the actual reaction # = index + 1
    orderstr_1600.append(MyGasMech.getgasreactionstring(rxn_order[i]+1))

<h5>Find reaction rates at the raised temperature</h5>Raise the mixture temperature from 1600K to 1800K. You should expect that the absolute values of the net reaction rates would increase accordingly.

In [ ]:
# change the mixture temperature
premixed.temperature = 1800.0

Use the <code>listreactionrates</code> method to obtain the net reaction rates at 1800K.

In [ ]:
# get the list the nonzero net reaction rates at the new temperature
rxn_order, net_rxn_rates_1800 = premixed.listreactionrates()

Convert the reaction index in <code>rxn_order</code> as described in <strong>Step four</strong> and save it to <code>orderstr_1800</code>. 

In [ ]:
# convert reaction # from integers to strings
orderstr_1800 = []
for i in range(len(rxn_order)):
    # the array index starting from 0 so the actual reaction # = index + 1
    orderstr_1800.append(MyGasMech.getgasreactionstring(rxn_order[i]+1))


<h5>Isolate the major reactions and compare the reaction rates</h5>Make a horizontal bar plot to find the dominant reactions at each temperature and compare their net reaction rates.

In [ ]:
# create a rate plot
plt.rcParams.update({'figure.autolayout': True})
plt.subplots(2, 1, sharex='col', figsize=(6, 5))
# use horizontal bar chart
plt.subplot(211)
plt.barh(orderstr_1600, net_rxn_rates_1600, color='blue', height=0.4)
# use log scale on x axis
plt.xscale('symlog')
plt.ylabel('reaction #')
plt.text(-3.0e-4, 0.5, 'T = 1600K', fontsize=10)
#
plt.subplot(212)
plt.barh(orderstr_1800, net_rxn_rates_1800, color='orange', height=0.4)
plt.xlabel('reaction rate [mole/cm3-sec]')
plt.ylabel('reaction #')
plt.text(-3.0e-4, 0.5, 'T = 1800K', fontsize=10)
# use log scale on x axis
plt.xscale('symlog')